In [42]:
import swmmAPI as swmm

import numpy as np


headers = [
    '[TITLE]',
    '[OPTIONS]',
    '[EVAPORATION]',
    '[RAINGAGES]',
    '[SUBCATCHMENTS]',
    '[SUBAREAS]',
    '[INFILTRATION]',
    '[JUNCTIONS]',
    '[OUTFALLS]',
    '[STORAGE]',
    '[CONDUITS]',
    '[PUMPS]',
    '[ORIFICES]',
    '[WEIRS]',
    '[XSECTIONS]',
    '[LOSSES]',
    '[CONTROLS]',
    '[INFLOWS]',
    '[DWF]',
    '[HYDROGRAPHS]',
    '[RDII]',
    '[CURVES]',
    '[TIMESERIES]',
    '[PATTERNS]',
    '[REPORT]',
    '[TAGS]',
    '[MAP]',
    '[COORDINATES]',
    '[VERTICES]',
    '[Polygons]',
    '[SYMBOLS]',
    '[PROFILES]'
]

inpF = 'NoWestside_withISDs_RainEvents.inp'

sections = swmm.make_sections(inpF,headers)

maps,coords,verts,polygons = swmm.get_geo_dicts(sections)
conversion = 3.2808398950 # Feet to Meters
coords = swmm.convert(coords,conversion)
verts = swmm.convert(verts,conversion)
polygons = swmm.convert(polygons,conversion)
conduitDict,junctionDict,storageDict,subcatchmentDict,outfallDict,orificeDict,curvesDict,pumpDict,optionsDict,raingauges,timeseries,evaporation,infiltration,losses,controls,inflows = swmm.prep_dicts(sections)

In [12]:
raingauges = {}
for l in sections['[RAINGAGES]']:
    a = l.split()
    raingauges[a[0]] = {
        'format' : a[1],
        'interval' : float(a[2]),
        'SCF' : float(a[3]),
        'source1' : a[4],
        'source2' : a[5]
    }

In [18]:
timeseries = {}

for l in sections['[TIMESERIES]']:
    a = l.split()
    
    try:
        timeseries[a[0]]['time'].append(a[1])
        timeseries[a[0]]['value'].append(a[2])
    except:
        timeseries[a[0]] = {
            'time' : [a[1]],
            'value' : [a[2]]
        }

In [28]:
evap_raw = sections['[EVAPORATION]']
evaporation = {
    'constant' : float(evap_raw[0].split()[1]),
    'dry_only' : evap_raw[1].split()[1]    
}
del evap_raw

In [30]:
infiltration = {}
for l in sections['[INFILTRATION]']:
    a = l.split()
    
    infiltration[a[0]] = {
        'max_rate' : float(a[1]),
        'min_rate' : float(a[2]),
        'decay' : float(a[3]),
        'dry_time' : float(a[4]),
        'max_infil' : float(a[5])
    }

In [32]:
losses = {}
for l in sections['[LOSSES]']:
    a = l.split()
    
    losses[a[0]] = {
        'k_entry' : float(a[1]),
        'k_exit' : float(a[2]),
        'k_avg' : float(a[3]),
        'flap_gate' : a[4],
        'seepage' : float(a[5])
    }

In [43]:
control_list = [l for l in sections['[CONTROLS]']]
controls = {}
for i in control_list:
    if 'RULE' in i:
        controls[i.split()[1]] = []

for i in control_list:
    if 'RULE' in i:
        rule = i.split()[1]
    else:
        controls[rule].append(i)

In [50]:
inflows = {}
for l in sections['[INFLOWS]']:
    a = l.split()
    
    inflows[a[0]] = {
        'constituent' : a[1],
        'time_series' : a[2],
        'type' : a[3],
        'm_factor' : float(a[4]),
        's_factor' : float(a[5])
    }
    
    try:
        a[0]['baseline'] = a[6]
        a[0]['pattern'] = a[7]
    except:
        pass

In [2]:
def raingauges_from_subcatchment(subcatchmentDict,raingauges,rg_format,interval,SCF,source1):
    for sub in subcatchmentDict:
        subcatchmentDict[sub]['rain_gage'] = sub
        
        raingauges[sub] = {
            'format' : rg_format,
            'interval' : interval,
            'SCF' : SCF,
            'source1' : source1,
            'source2' : sub
        }

In [3]:
def timeseries_from_raingauges(raingauges,timeseries):
    for sub in subcatchmentDict:
        try: 
            if timeseries[sub]:
                pass
#                 print(sub + ' exists')

        except:
            timeseries[sub] = {
                'Date': [],
                'Time': [],
                'Value': [],
            }

def add_timeseries_info(timeseries,name,date,time,value):
    timeseries[name]['Date'].append(date)
    timeseries[name]['Time'].append(time)
    timeseries[name]['Value'].append(value)

In [4]:
raingauges_from_subcatchment(subcatchmentDict,raingauges,'INTENSITY',0.25,1.0,'TIMESERIES')
timeseries_from_raingauges(raingauges,timeseries)

In [7]:
for name in timeseries:
    # Check on if the raingauge/timeseries is a direct link to the subcatchment
    for i in range(0,5):
        if name in subcatchmentDict.keys(): 
            # input specific values into the timeseries... incorporate other processes here.
            date = ''
            time = i * 0.25
            value = i * 0.1
            add_timeseries_info(timeseries,name,date,time,value)

In [9]:
def write_timeseries(timeseries,sublist):
    with open('timeseries.txt','w') as f:
        f.write('[TIMESERIES]\n')
        f.write(';;Name           Date       Time       Value     \n')
        f.write(';;-------------- ---------- ---------- ----------\n')
        
        for ts in timeseries:
            if ts in sublist:
                wrt = timeseries[ts]
                for d,t,v in zip(wrt['Date'],wrt['Time'],wrt['Value']):
                    f.write(ts + '\t \t' + str(d) + '\t \t' + str(t) + '\t \t' + str(v) + '\n')
                    
                f.write(';\n')
    f.close()

In [23]:
def write_raingauges(raingauges,sublist):
    with open('raingauges.txt','w') as f:
        f.write('[RAINGAGES]\n')
        f.write(';;Name           Format    Interval SCF      Source \n')
        f.write(';;-------------- --------- ------ ------ ----------\n')
        
        for gage in raingauges:
            if gage in sublist:
                rg = raingauges[gage]
                f.write(gage + '\t\t\t' + str(rg['format']) + '\t' + str(rg['interval']) + '\t' + str(rg['SCF']) + '\t' + str(rg['source1']) + '\t' + str(rg['source2']) + '\n')

In [39]:
def write_subcatchments(subcatchmentDict):
    with open('subcatchments.txt','w') as f:
        f.write('[SUBCATCHMENTS]\n')
        f.write(';;Name           Rain Gage        Outlet           Area     %Imperv  Width    %Slope   CurbLen  SnowPack\n')
        f.write(';;-------------- ---------------- ---------------- -------- -------- -------- -------- -------- ----------------\n')
        
        for sub in subcatchmentDict:
            s = subcatchmentDict[sub]
            f.write(sub + '\t\t' + s['rain_gage'] + '\t\t' + s['outlet'] + '\t\t' + str(s['area']) + '\t' + str(s['per_imperv']) + '\t' + str(s['width']) + '\t' + str(s['slope']) + '\t' + str(s['curblen']) + '\n')
        

In [29]:
subcatchmentDict['S1000']

{'rain_gage': 'S1000',
 'outlet': 'RC11000',
 'area': 213.7,
 'per_imperv': 72.8,
 'width': 93088.0,
 'slope': 1.0,
 'curblen': 0.0}

In [17]:
write_timeseries(timeseries,subcatchmentDict.keys())

In [25]:
write_raingauges(raingauges,subcatchmentDict.keys())

In [40]:
write_subcatchments(subcatchmentDict)

In [27]:
for sub in subcatchmentDict:
    subcatchmentDict[sub]['rain_gage'] = sub

In [5]:
from pyproj import Proj, transform
from shapely.geometry import Polygon
from descartes import PolygonPatch

inputEPSG = Proj(init='epsg:6498')
outputEPSG = Proj(init='epsg:4326') # WGS84

for key in polygons:
    polygons[key]['lat'] = []
    polygons[key]['lon'] = []
    polygons[key]['shapely'] = []
    for x,y in zip(polygons[key]['x'],polygons[key]['y']):
        lat,lon = transform(inputEPSG,outputEPSG,x,y)
        
        polygons[key]['lat'].append(lat)
        polygons[key]['lon'].append(lon)
        polygons[key]['shapely'].append((lat,lon))
    
    polygons[key]['polygon'] = Polygon(polygons[key]['shapely']) 

In [3]:
import pickle

pickleOut = 'NoWestide_geo.p'

saveDict = {
    'polygons': polygons,
    'coords' : coords,
    'maps' : maps,
    'verts' : verts,
}

with open(pickleOut,'wb') as f:
    pickle.dump(saveDict,f)

In [25]:
shape = Polygon(polygons[key]['shapely'])
shape.bounds

(-83.21442564842441, 42.504508896212506, -83.11112529164289, 42.56078390964576)

{'S1000': {'x': [4102779.602416411,
   4102737.235216411,
   4102727.481616411,
   4102627.5072164102,
   4102599.77041641,
   4102497.9672164097,
   4102494.6144164097,
   4102447.0656164098,
   4102441.5792164095,
   4102300.456816409,
   4102171.8312164084,
   4102147.4472164083,
   4102015.468816408,
   4101945.3648164077,
   4101936.5256164074,
   4101821.920816407,
   4101632.9448164064,
   4101624.105616406,
   4101570.156016406,
   4101498.8328164057,
   4101454.6368164057,
   4101547.296016406,
   4101676.8360164063,
   4101840.8184164073,
   4102031.422658008,
   4102063.286450008,
   4102849.068165211,
   4103016.451828412,
   4103040.498110012,
   4102935.3552164114,
   4102874.0904164114,
   4102808.2536164112,
   4102779.602416411],
  'y': [88351.86180275341,
   88227.04620275291,
   88197.84240035279,
   88005.93270035202,
   87971.46164915188,
   87845.40795155137,
   87841.77900275137,
   87789.62955155114,
   87782.68559795112,
   87603.08250275042,
   87513.185400350